* Strona biblioteki: https://scikit-learn.org
* Dokumentacja/User Guide: https://scikit-learn.org/stable/user_guide.html

In [1]:
import numpy as np
import pandas as pd
import sklearn

sklearn.__version__

'1.7.1'

In [2]:
# słownik z danymi
data = {
    "size": ["XL", "L", "M", "L", "M"],
    "color": ["red", "green", "blue", "green", "red"],
    "gender": ["female", "male", "male", "female", "female"],
    "price": [199.0, 89.0, 99.0, 129.0, 79.0],
    "weight": [500, 450, 300, 380, 410],
    "bought": ["yes", "no", "yes", "no", "yes"],
}
# do data frame z pandas bo taki typ przyjmuje scikit
df_raw = pd.DataFrame(data=data)
df_raw

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500,yes
1,L,green,male,89.0,450,no
2,M,blue,male,99.0,300,yes
3,L,green,female,129.0,380,no
4,M,red,female,79.0,410,yes


In [3]:
# kopia do pracy
df = df_raw.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   size    5 non-null      object 
 1   color   5 non-null      object 
 2   gender  5 non-null      object 
 3   price   5 non-null      float64
 4   weight  5 non-null      int64  
 5   bought  5 non-null      object 
dtypes: float64(1), int64(1), object(4)
memory usage: 372.0+ bytes


In [4]:
# zamiana całych kolumn na inne typy
df["weight"] = df["weight"].astype("float")  # zamian na typ float

for col in ["size", "color", "gender", "bought"]:  # zmiana na typ category, do statystyk i do późniejszych konwersji
    df[col] = df[col].astype("category")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   size    5 non-null      category
 1   color   5 non-null      category
 2   gender  5 non-null      category
 3   price   5 non-null      float64 
 4   weight  5 non-null      float64 
 5   bought  5 non-null      category
dtypes: category(4), float64(2)
memory usage: 744.0 bytes


In [5]:
df.describe() # podstawowe statystyki obiektu tylko dla kolumn numerycznych

,price,weight
count,5.000000,5.000000
mean,119.000000,408.000000
std,48.476799,75.299402
min,79.000000,300.000000
25%,89.000000,380.000000
50%,99.000000,410.000000
75%,129.000000,450.000000
max,199.000000,500.000000


In [6]:
df.describe().T # trasponowanie czyli kopnięcie w tabelke (ilość, średnia, odchylenie standardowe, minimalna, kwantyle, maksymalna)

,count,mean,std,min,25%,50%,75%,max
price,5.0,119.0,48.476799,79.0,89.0,99.0,129.0,199.0
weight,5.0,408.0,75.299402,300.0,380.0,410.0,450.0,500.0


In [7]:
df.describe(include="category").T 

,count,unique,top,freq
size,5,3,L,2
color,5,3,green,2
gender,5,2,female,3
bought,5,2,yes,3


### Zamiana danych na dane numeryczne

In [8]:
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,yes
1,L,green,male,89.0,450.0,no
2,M,blue,male,99.0,300.0,yes
3,L,green,female,129.0,380.0,no
4,M,red,female,79.0,410.0,yes


### LabelEncoder - wykorzystywany do danych wyjściowych klasyfikacji binarnej (sugestja, dobra praktyka :))

In [9]:
from sklearn.preprocessing import LabelEncoder

In [10]:
# zamiana kolumny "bought" na 1,0 zamiast yes,no
le = LabelEncoder()
le.fit(df['bought'])
le.transform(df['bought'])

array([1, 0, 1, 0, 1])

In [11]:
# to samo co wyżej tylko w wersji skróconej
le.fit_transform(df['bought'])

array([1, 0, 1, 0, 1])

In [12]:
# podgląd sposobu mapowania - miejsce 0 to 'no', miejsce 1 to 'yes'
le.classes_

array(['no', 'yes'], dtype=object)

In [13]:
# a teraz wszystko naraz włacznie z podmianą
df['bought'] = le.fit_transform(df['bought'])
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,1
1,L,green,male,89.0,450.0,0
2,M,blue,male,99.0,300.0,1
3,L,green,female,129.0,380.0,0
4,M,red,female,79.0,410.0,1


In [14]:
# powrót do oryginalnej postaci
df['bought'] = le.inverse_transform(df['bought'])
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,yes
1,L,green,male,89.0,450.0,no
2,M,blue,male,99.0,300.0,yes
3,L,green,female,129.0,380.0,no
4,M,red,female,79.0,410.0,yes


### OneHotEncoder

In [15]:
from sklearn.preprocessing import OneHotEncoder

In [16]:
encoder = OneHotEncoder(sparse_output=False) # wyłaczenie zwrotu macierzy rzadkiej (przechowywanie w pamięci macieży tylko z pozycjami jedynek)
encoder.fit(df[['size']])

,categories,'auto'
,drop,None
,sparse_output,False
,dtype,<class 'numpy.float64'>
,handle_unknown,'error'
,min_frequency,None
,max_categories,None
,feature_name_combiner,'concat'


In [17]:
encoder.transform(df[['size']]) # rozłożenie kolumny size na trzy kolumny L M XL, w wierszach jak występuje to jest 1

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [18]:
encoder.categories_ # rozpiska kolumn do ww macierzy

[array(['L', 'M', 'XL'], dtype=object)]

In [19]:
# wszystko naraz
encoder = OneHotEncoder(drop='first', sparse_output=False) # drop='first'usunięcie liniowej zalerzności 
encoder.fit(df[['size']])
encoder.transform(df[['size']])

array([[0., 1.],
       [0., 0.],
       [1., 0.],
       [0., 0.],
       [1., 0.]])

In [20]:
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,yes
1,L,green,male,89.0,450.0,no
2,M,blue,male,99.0,300.0,yes
3,L,green,female,129.0,380.0,no
4,M,red,female,79.0,410.0,yes


### PANDAS get_dummies() - bardziej elastycha niż poprzednia

In [21]:
# przy pomocy pandas, jest szybciej i krócej
pd.get_dummies(data=df)

,price,weight,size_L,size_M,size_XL,color_blue,color_green,color_red,gender_female,gender_male,bought_no,bought_yes
0,199.0,500.0,False,False,True,False,False,True,True,False,False,True
1,89.0,450.0,True,False,False,False,True,False,False,True,True,False
2,99.0,300.0,False,True,False,True,False,False,False,True,False,True
3,129.0,380.0,True,False,False,False,True,False,True,False,True,False
4,79.0,410.0,False,True,False,False,False,True,True,False,False,True


In [22]:
pd.get_dummies(data=df, drop_first=True, dtype=int) # drop_first usunięcie liniowej zalerzności 

,price,weight,size_M,size_XL,color_green,color_red,gender_male,bought_yes
0,199.0,500.0,0,1,0,1,0,1
1,89.0,450.0,0,0,1,0,1,0
2,99.0,300.0,1,0,0,0,1,1
3,129.0,380.0,0,0,1,0,0,0
4,79.0,410.0,1,0,0,1,0,1


In [23]:
pd.get_dummies(data=df, drop_first=True, dtype=int, columns=['size']) # koduje kolumny które chcemy

,color,gender,price,weight,bought,size_M,size_XL
0,red,female,199.0,500.0,yes,0,1
1,green,male,89.0,450.0,no,0,0
2,blue,male,99.0,300.0,yes,1,0
3,green,female,129.0,380.0,no,0,0
4,red,female,79.0,410.0,yes,1,0


## Standaryzacja, Skalowanie (StandardScaler)
* pandas std() - domyślnie estymator nieobciążony wariancji
* numpy std() - domyślnie estymator obciążony wariancji
### Kiedy używać którego ustawienia?
* pandas.std() (lub numpy.std(ddof=1)) gdy analizujesz próbę danych, z której chcesz wnioskować o większej populacji (np. wyniki ankiety, pomiary eksperymentalne). Jest to standard w statystyce inferencyjnej.
* numpy.std() (lub pandas.std(ddof=0)) gdy operujesz na kompletnej populacji (np. wszystkie oceny uczniów w jednej klasie, wyniki wszystkich meczów w sezonie) lub w niektórych algorytmach uczenia maszynowego/matematycznych, gdzie wymagana jest spójność.

In [24]:
# Liczenie na piechote średniej i odchylenia std - pandas
print(f"{df['price']}\n")  # wyświetlenie kolumny
print(f" Średnia : {df['price'].mean()}\n")
print(f" Odchylenie standardowe : {df['price'].std()}\n")

0    199.0
1     89.0
2     99.0
3    129.0
4     79.0
Name: price, dtype: float64

 Średnia : 119.0

 Odchylenie standardowe : 48.47679857416329



In [25]:
# Liczenie na piechote średniej i odchylenia std - numpy
tab = df["price"].to_numpy()
print(f"{tab}\n")  # wyświetlenie kolumny
print(f" Średnia : {tab.mean()}\n")
print(f" Odchylenie standardowe : {tab.std()}\n")

[199.  89.  99. 129.  79.]

 Średnia : 119.0

 Odchylenie standardowe : 43.3589667773576



In [26]:
# funkcja skalująca 
def standardize(x):
    return (x - x.mean()) / x.std()

standardize(df['price'])

0    1.650274
1   -0.618853
2   -0.412568
3    0.206284
4   -0.825137
Name: price, dtype: float64

In [27]:
standardize(tab)

array([ 1.84506242, -0.69189841, -0.4612656 ,  0.2306328 , -0.92253121])

## Skalowania na SciKit

In [28]:
from sklearn.preprocessing import scale # szybka funkcja

In [29]:
scale(df['price'])

array([ 1.84506242, -0.69189841, -0.4612656 ,  0.2306328 , -0.92253121])

In [30]:
from sklearn.preprocessing import StandardScaler # pełny obiekt skalowania z zapamiętaniem danych statystycznych

In [31]:
scaler = StandardScaler()
scaler.fit(df[['price']])
scaler.transform(df[['price']])

array([[ 1.84506242],
       [-0.69189841],
       [-0.4612656 ],
       [ 0.2306328 ],
       [-0.92253121]])

In [32]:
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,yes
1,L,green,male,89.0,450.0,no
2,M,blue,male,99.0,300.0,yes
3,L,green,female,129.0,380.0,no
4,M,red,female,79.0,410.0,yes


In [33]:
scaler = StandardScaler()
df[['price', 'weight']] = scaler.fit_transform(df[['price', 'weight']])
df

,size,color,gender,price,weight,bought
0,XL,red,female,1.845062,1.366002,yes
1,L,green,male,-0.691898,0.623610,no
2,M,blue,male,-0.461266,-1.603567,yes
3,L,green,female,0.230633,-0.415740,no
4,M,red,female,-0.922531,0.029696,yes


# Przygotowanie w całości od początku do konca

In [34]:
df = df_raw.copy();
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500,yes
1,L,green,male,89.0,450,no
2,M,blue,male,99.0,300,yes
3,L,green,female,129.0,380,no
4,M,red,female,79.0,410,yes


In [35]:
le = LabelEncoder()
df['bought'] = le.fit_transform(df['bought'])

scaler = StandardScaler()
df[['price', 'weight']] = scaler.fit_transform(df[['price', 'weight']])

df = pd.get_dummies(data=df, drop_first=True, dtype=int)

df

,price,weight,bought,size_M,size_XL,color_green,color_red,gender_male
0,1.845062,1.366002,1,0,1,0,1,0
1,-0.691898,0.623610,0,0,0,1,0,1
2,-0.461266,-1.603567,1,1,0,0,0,1
3,0.230633,-0.415740,0,0,0,1,0,0
4,-0.922531,0.029696,1,1,0,0,1,0


In [36]:
# przesunięcie kolumny wyjściowej na koniec
col = df.pop('bought')
df.insert(len(df.columns), col.name, col)
df

,price,weight,size_M,size_XL,color_green,color_red,gender_male,bought
0,1.845062,1.366002,0,1,0,1,0,1
1,-0.691898,0.623610,0,0,1,0,1,0
2,-0.461266,-1.603567,1,0,0,0,1,1
3,0.230633,-0.415740,0,0,1,0,0,0
4,-0.922531,0.029696,1,0,0,1,0,1
